## Esercizio 51 - Produzione della pasta


## Describe the business problem

This notebook describes how to use CPLEX Modeling for Python to manage the production of pasta to meet demand with your resources.

The model aims at minimizing the production cost for a number of products while satisfying customer demand.

   * Each product can be produced either inside the company or outside, at a higher cost.
   * The inside production is constrained by the company's resources, while outside production is considered unlimited.

The model first declares the products and the resources.
The data consists of the description of the products (the demand, the inside and outside costs, and the resource consumption) and the capacity of the various resources.
The variables for this problem are the inside and outside production for each product.

## Use decision optimization

* Data:

> $P$ set of products

> $R$ set of resources

> $c_{p,r}$ consumption of resource $r$ for each product $p$ 

> $d_p$ customer demand of product $p$

> $\alpha_p$ inside cost for each product $p$

> $\beta_p$ outside cost for each product $p$

> $\gamma_r$ inside capacity for resource $r$

* Variables:

> $i_p$ inside production for product $p$

> $o_p$ outside production for product $p$

\begin{align}
\min \quad&\sum\limits_{p \in P} \alpha_p \ i_p + \beta_p \ o_p &\\
s.t.\quad&i_p + o_p \geq d_p & p \in P,\\
\quad&\sum\limits_{p \in P} i_p \ c_{p,r} \leq \gamma_r & r \in R,\\
\quad&i_p \geq 0 &p \in P\\
\quad&o_p \geq 0 &p \in P
\end{align}



* TEST: pasta.json

> kluski inside_production = 40.0 outside_production = 60.0<br>
> capellini inside_production = 0 outside_production = 200.0<br>
> fettucine inside_production = 0 outside_production = 300.0<br>
> Total inside cost = 24.0, total outside cost = 348.0<br>
> Total cost = 372

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/pasta.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import json
import matplotlib.pyplot as plt
import docplex.mp.model as cplex

with open('pasta.json', 'r') as file:
    dati = json.load(file)


n = len(dati['PRODUCTS']) #numero di prodotti
m = len(dati['RESOURCES']) #numero di risorse
P = list(range(n))
R = list(range(m))

# info sui prodotti
alfa = []
beta = []
d = []
nomi = []

for p in dati['PRODUCTS']:
    alfa.append(p['inside_cost'])
    beta.append(p['outside_cost'])
    d.append(p['demand'])
    nomi.append(p['name'])
# info sulle risorse
gamma = []

#riempire i vettori
for r in dati['RESOURCES']:
    gamma.append(r['capacity'])

# matrice prodotti x risorse
c = dati['CONSUMPTIONS']

with cplex.Model('pasta') as mdl:
    i = mdl.continuous_var_list(P, name='i')
    o = mdl.continuous_var_list(P, name='o')
    
    mdl.minimize(sum(alfa[p]*i[p]+beta[p]*o[p] for p in P))
    
    for p in P:
        mdl.add_constraint(i[p] + o[p] >= d[p])
    
    for r in R:
        mdl.add_constraint(sum(i[p]*c[p][r] for p in P) <= gamma[r])
    
    mdl.print_information()
    sol = mdl.solve()
    
    if sol == None:
        print('NO solution')
    else:
        print(sol)
        
        print('Total cost:' + str(sol.objective_value))
        
        total_inside_cost = 0
        total_outside_cost = 0
        for p in P:
            print('Prodotto {} inside_production={} outside_production={}'.format(nomi[p], i[p].solution_value, o[p].solution_value))
            total_inside_cost += i[p].solution_value * alfa[p]
            total_outside_cost += o[p].solution_value * beta[p]
        
        #total_inside_cost = sum(alfa[p]*i[p].solution_value for p in P)
        #total_outside_cost = sum(beta[p]*o[p].solution_value for p in P)
        print('Total inside cost {}, total outside cost {}'.format(total_inside_cost, total_outside_cost))
        
        
        inside_values = [i[p].solution_value for p in P]
        outside_values = []
        for p in P:
            outside_values.append(o[p].solution_value)
        
        plt.pie(inside_values, labels=nomi, autopct='%1.1f%%')
        plt.title('Inside production')
        plt.show()
        
        plt.pie(outside_values, labels=nomi, autopct='%1.1f%%')
        plt.title('Outside production')
        plt.show()
        
        